The code in `pyldl.applications.lesion_counting` is basically an unofficial implementation of the paper "Joint Acne Image Grading and Counting via Label Distribution Learning" (*ICCV* 2019).

In [1]:
import pyldl.applications.lesion_counting as lc
from pyldl.utils import LDLEarlyStopping

In [2]:
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD

In [3]:
X, grades, counts = lc.load_acne04('./ACNE04')

In [4]:
n_grades = int(max(grades) + 1)
n_counts = int(max(counts))

In [5]:
D = lc.preprocessing(counts, sigma=3., n_counts=n_counts)

In [6]:
X_train, X_val, D_train, D_val = train_test_split(X, D, test_size=.1, random_state=0)

In [7]:
model = lc.LDL_ACNE(n_grades, n_counts)

In [8]:
model.fit(X_train, D_train, verbose=1, batch_size=32, X_val=X_val, D_val=D_val, optimizer=SGD(1e-5),
          callbacks=[LDLEarlyStopping(monitor='mean_absolute_error', patience=None)]);

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4059s 4s/step - loss: 7.3778 - mean_absolute_error: 4.9891


In [9]:
X_test, grades_test, counts_test = lc.load_acne04('./ACNE04', mode='test')

In [10]:
scores = model.score(X_test, grades_test,
                     mode='grading', batch_size=32, return_dict=True)
for (k, v) in scores.items():
    print(f"{k}: {v:.4f}")

precision: 0.7403
specificity: 0.8956
sensitivity: 0.6979
youden_index: 0.5935
accuracy: 0.7363


In [11]:
import numpy as np

In [12]:
scores = model.score(X_test, counts_test,
                     mode='counting', batch_size=32, return_dict=True)
s = list(scores.items())
print(f"{s[0][0]}: {s[0][1]:.4f}")
print(f"(root) {s[1][0]}: {np.sqrt(s[1][1]):.4f}")

mean_absolute_error: 4.5171
(root) mean_squared_error: 7.0148
